In [112]:
import lxml.html
import requests
import re
import os
import time
import datetime
import pandas as pd
from pprint import pprint
import plotly.express as px
import plotly.graph_objects as go

In [3]:
print("Pandas:",pd.__version__)

Pandas: 1.2.3


## Functions

In [104]:
def get_details(urls):
    """
    input:
        - urls: generator object, 詳細ページのurlを生成するgenerator
    output: usedcars DF
    """
    carsDF = None
    for url in urls:
        key = extract_key(url)    
        usedcar = scrape_detail_page(url, key)
        try:
            df_ = pd.DataFrame(usedcar,index=[0])
        except Exception as e:
            print(e)
            print("Error converting dict to DF")
            pprint(usedcar)
            return None
        if carsDF is None:
            carsDF = df_
        else:
            carsDF = pd.concat([carsDF,df_],ignore_index=True)
    return carsDF
        


def get_lastpage_num(url):
    """
    リストページのhtmlからリストページの総数を取得して返す
    intput:
        - url: str
    output:
        - last_num: int, リストページ数
    """
    response = requests.get(url)
    root = lxml.html.fromstring(response.content)
    # 最後のlistページのurlを取得する
    last_page = root.cssselect(CSS_LAST_PAGE_NUM)[0]
    # urlから最後のindex番号を取得する
    last_index = last_page.get('href').split("/")[-1]
    m = re.search('index([0-9]+)', last_index)
    # re.search()でマッチしなかった場合のエラー処理
    if m is None:
        raise ValueError('index.html of last page is not found')
    last_num = int(m.group(1))
    return last_num


def scrape_list_page(url):
    """
    リストページから詳細ページのurlを取得し、生成する(generator object)
    input:
        - url: str, リストページのurl
    output:
        generator object, 詳細ページのurlを生成する
    """
    # webページ読み込み、response.contentから文字列を取得してパース
    response = requests.get(url)
    root = lxml.html.fromstring(response.content)
    # 全てのurlを絶対urlに変換
    root.make_links_absolute(response.url)
    htmlElems = root.cssselect(CSS_DETAIL_PAGE_URL)
    for elem in htmlElems:
        url = elem.get('href')
        yield url


def scrape_detail_page(url, key):
    """
    詳細ページから各種情報を取得し、dictで返す
    input:
        - url: str, 詳細ページのurl
        - key: str, 詳細ページのurlに対するキー
    output:
        - usedcar: dict, dbに格納するドキュメント
    """
    true_mark = "◯"

    response = requests.get(url)
    root = lxml.html.fromstring(response.content)

    # 本体価格---------------------------------------------------------------
    base_price = root.cssselect(CSS_BODY_PRICE)
    base_price = base_price[0].get('content').replace(',', '')
    base_price = int(base_price)
    # 本体価格応相談の場合はNullにする
    if base_price > 90000000:
        base_price = None
    # 支払総額---------------------------------------------------------------
    total_price = root.cssselect(CSS_TOTAL_PRICE)
    total_price = total_price[0].text_content()
    total_price = re.sub(r'\n(\t)+', '', total_price).replace('万円', '')
    try:
        total_price = int(float(total_price) * 10000)
    except ValueError:
        total_price = None
    # 年式---------------------------------------------------------------
    model_year = root.cssselect(CSS_MODEL_YEAR)
    model_year = model_year[0].text
    model_year = int(model_year)
    # 走行距離---------------------------------------------------------------
    distance = root.cssselect(CSS_DISTANCE)
    distance = distance[0].text_content()
    distance = re.sub(r'\n(\t)+', '', distance).strip().replace('走行距離', '')
    if '万' in distance:
        distance = distance.replace('万km', '')
        try:
            distance = float(distance)
        except ValueError:
            distance = None
    else:
        distance = distance.replace('km', '')
        try:
            distance = float(distance) / 10000
        except ValueError:
            distance = None
    # 車検---------------------------------------------------------------
    inspection = root.cssselect(CSS_INSPECTION)
    inspection = inspection[0].text_content()
    inspection = re.sub(r'\n(\t)+', '', inspection).replace('車検有無', '')
    # 修復歴---------------------------------------------------------------
    repare = root.cssselect(CSS_REPARE)
    repare = repare[0].text
    # 本体情報等---------------------------------------------------------------
    info = root.cssselect(CSS_CAR_INFO)
    info = info[0].text
    info = info.replace(EM_SPACE, ' ').replace(NBSP, ' ')
    # 駆動方式----------------------------------------------------------------
    try:
        drive = root.cssselect(CSS_DRIVE)
        drive = drive[0].text
    except IndexError:
        css_drive_tmp = CSS_DRIVE.replace(
            'section:nth-child(5)', 'section:nth-child(4)'
        )
        drive = root.cssselect(css_drive_tmp)
        try:
            drive = drive[0].text
        except IndexError:            
            drive=''
    # リサイクル料-------------------------------------------------------------
    recycle = root.cssselect(CSS_RECYCLE)
    recycle = recycle[0].text
    # 法定整備---------------------------------------------------------------
    legal_maintenance = root.cssselect(CSS_LEGAL_MAINTENANCE)
    legal_maintenance = legal_maintenance[0].text
    # 保証---------------------------------------------------------------
    warranty = root.cssselect(CSS_WARRANTY)
    warranty = warranty[0].text
    warranty = warranty.replace(EM_SPACE, ' ')
    # ワンオーナー------------------------------------------------------------
    one_owner = root.cssselect(CSS_ONE_OWNER)
    one_owner = one_owner[0].text
    if one_owner == true_mark:
        one_owner = 1
    else:
        one_owner = 0
    # 定期点検記録簿-----------------------------------------------------------
    record_book = root.cssselect(CSS_RECORD_BOOK)
    record_book = record_book[0].text
    if record_book == true_mark:
        record_book = 1
    else:
        record_book = 0
    # 禁煙車---------------------------------------------------------------
    no_smoke = root.cssselect(CSS_NO_SMOKE)
    no_smoke = no_smoke[0].text
    if no_smoke == true_mark:
        no_smoke = 1
    else:
        no_smoke = 0
    # 地域-----------------------------------------------------------------
    region = root.cssselect(CSS_REGION)
    region = region[0].text

    usedcar = {
        'url': response.url,
        'base_price': base_price,
        'total_price': total_price,
        'model_year': model_year,
        'distance': distance,
        'repare': repare,
        'inspection': inspection,
        'info': normalize_spaces(info),
        'drive': drive,
        'recycle': recycle,
        'legal_maintenance': legal_maintenance,
        'warranty': warranty,
        'one_owner': one_owner,
        'record_book': record_book,
        'no_smoke': no_smoke,
        'key': key,
        'region': region
    }

    # オプション情報を取得してusedcarに追加
    options = get_options_state(root)
    usedcar.update(options)
    return usedcar


def get_options_state(root):
    """
    車両のオプション情報を取得する
    input:
        - root: lxml.html.
    """
    # オプション一覧を辞書にまとめる
    options = {
        'keyless': CSS_KEYLESS,
        'smartkey': CSS_SMARTKEY,
        'navi': CSS_NAVI,
        'TV': CSS_TV,
        'video': CSS_VIDEO,
        'audio': CSS_AUDIO,
        'player': CSS_PLAYER,
        'monitor': CSS_MONITOR,
        'ETC': CSS_ETC,
        'sheat_air': CSS_SHEAT_AIR,
        'sheat_heater': CSS_SHEAT_HEATER,
        'idling_stop': CSS_IDLINGSTOP,
        'AS_sensor': CSS_AS_SENSOR,
        'cruise': CSS_CRUISE,
        'ABS': CSS_ABS,
        'ESC': CSS_ESC,
        'anti_theft': CSS_ANTI_THEFT,
        'auto_brake': CSS_AUTO_BRAKE,
        'parking_assist': CSS_PARKING_ASSIST,
        'airbag': CSS_AIRBAG,
        'headlight': CSS_HEADLIGHT,
        'camera': CSS_CAMERA,
        'around_camera': CSS_AROUND_CAMERA,
        'aero': CSS_AERO,
        'alumi_wheel': CSS_ALUMI_WHEEL,
        'lowdown': CSS_LOWDOWN,
        'liftup': CSS_LIFTUP,
        'cold_area': CSS_COLD_AREA
    }
    result = {}
    for key, option in options.items():
        htmlElems = root.cssselect(option)
        if len(htmlElems) == 0:
            option = option.replace(
                'section:nth-child(6)', 'section:nth-child(7)'
            )
            htmlElems = root.cssselect(option)
        if len(htmlElems) == 0:
            option = option.replace(
                'section:nth-child(7)', 'section:nth-child(5)'
            )
            htmlElems = root.cssselect(option)
        result[key] = get_option_value(key, htmlElems)
    return result


def get_option_value(key, elem):
    """
    option辞書に入れるvalue値を取得する
    input:
        - key: str, オプション名
        - elem: 詳細ページのlxml.html.HTMLElemオブジェクト
    output:
        - value: int or str, option辞書のkeyに対するvalue
    """
    class_id = elem[0].attrib['class']
    if 'active' in class_id:
        text = elem[0].text
        if key in ['navi', 'video', 'audio', 'airbag', 'camera']:
            value = text.split('：')[-1]
        elif key == 'TV':
            if 'フルセグ' in text:
                value = 'フルセグ'
            else:
                value = 'ワンセグ'
        elif key == "headlight":
            if 'ディスチャージ' in text:
                value = 'ディスチャージドランプ'
            else:
                value = 'LED'
        else:
            value = 1
    else:
        value = 0
    return value


def normalize_spaces(s):
    """
    文字列内の連続する空白を一つにし、前後の空白を削除する
    input:
        - s: str
    output: str
    """
    return re.sub(r'\s+', ' ', s).strip()


def extract_key(url):
    """
    詳細ページのurlからキーを取得し、返す
    input:
        - url: str, 詳細ページのurl
    output: str
    """
    key = url.split('/')[-2]
#     if not key.startswith('CU'):
#         raise ValueError('key extract error, detail url is wrong')
    return key


# Constract URL for car parameters
def getURL(model, **kwargs):    
    baseurl = 'https://www.carsensor.net/usedcar/' + model_dict[model]
    if 'AR' in kwargs:
        ar = kwargs.pop('AR')
        baseurl += "a{}/".format(ar)

    search = []
    for k,v in parameters.items():
        search.append("{}={}".format(k,v))
    url = baseurl + "index.html?" + "&".join(search)
    return url

## Initialization

In [157]:
# 車両基本情報のCSS selector
CSS_BODY_PRICE = 'body > div.page > div:nth-child(6) > main > section > div > div.column__sub > div.priceWrap > div.basePrice > p.basePrice__price'
CSS_TOTAL_PRICE = 'body > div.page > div:nth-child(6) > main > section > div > div.column__sub > div.priceWrap > div.totalPrice > p.totalPrice__price'
CSS_MODEL_YEAR = 'body > div.page > div:nth-child(6) > main > section > div > div.column__sub > div.specWrap > div:nth-child(1) > p.specWrap__box__num'
CSS_DISTANCE = 'body > div.page > div:nth-child(6) > main > section > div > div.column__sub > div.specWrap > div:nth-child(2)'
CSS_INSPECTION = 'body > div.page > div:nth-child(6) > main > section > div > div.column__sub > div.specWrap > div:nth-child(3)'
CSS_REPARE = 'body > div.page > div:nth-child(6) > main > section > div > div.column__sub > div.specWrap > div:nth-child(4) > p:nth-child(2)'
CSS_CAR_INFO = 'body > div.page > div:nth-child(6) > main > section > h2 > span'
CSS_DRIVE = 'body > div.page > div:nth-child(8) > div > div.column__main > section:nth-child(5) > div > table > tbody > tr:nth-child(1) > td:nth-child(4)'
CSS_RECYCLE = 'body > div.page > div:nth-child(8) > div > div.column__main > section:nth-child(2) > div > table > tbody > tr:nth-child(6) > td:nth-child(2)'
CSS_LEGAL_MAINTENANCE = 'body > div.page > div:nth-child(8) > div > div.column__main > section:nth-child(2) > div > table > tbody > tr:nth-child(9) > td > p'
CSS_WARRANTY = 'body > div.page > div:nth-child(8) > div > div.column__main > section:nth-child(2) > div > table > tbody > tr:nth-child(10) > td > p'
CSS_ONE_OWNER = 'body > div.page > div:nth-child(8) > div > div.column__main > section:nth-child(2) > div > table > tbody > tr:nth-child(1) > td:nth-child(4)'
CSS_RECORD_BOOK = 'body > div.page > div:nth-child(8) > div > div.column__main > section:nth-child(2) > div > table > tbody > tr:nth-child(4) > td:nth-child(2)'
CSS_NO_SMOKE = 'body > div.page > div:nth-child(8) > div > div.column__main > section:nth-child(2) > div > table > tbody > tr:nth-child(5) > td:nth-child(2)'
CSS_REGION = 'body > div.page > div:nth-child(6) > main > section > div > div.column__sub > div.specWrap > div:nth-child(5) > p:nth-child(2)'

# 一覧ページのページ数を取得するCSS selector
CSS_LAST_PAGE_NUM = '#js-resultBar > div.resultBar__link > div > div.pager__text > a:nth-child(12)'
# 詳細ページのURL　CSS selector
CSS_DETAIL_PAGE_URL = '#carList h3[class="casetMedia__body__title"] > a'

# オプション CSS selector
CSS_KEYLESS = 'body > div.page > div:nth-child(8) > div > div.column__main > section:nth-child(6) > div > ul > li:nth-child(5)'
CSS_SMARTKEY = 'body > div.page > div:nth-child(8) > div > div.column__main > section:nth-child(6) > div > ul > li:nth-child(6)'
CSS_NAVI = 'body > div.page > div:nth-child(8) > div > div.column__main > section:nth-child(6) > div > ul > li:nth-child(7)'
CSS_TV = 'body > div.page > div:nth-child(8) > div > div.column__main > section:nth-child(6) > div > ul > li:nth-child(8)'
CSS_VIDEO = 'body > div.page > div:nth-child(8) > div > div.column__main > section:nth-child(6) > div > ul > li:nth-child(9)'
CSS_AUDIO = 'body > div.page > div:nth-child(8) > div > div.column__main > section:nth-child(6) > div > ul > li:nth-child(10)'
CSS_PLAYER = 'body > div.page > div:nth-child(8) > div > div.column__main > section:nth-child(6) > div > ul > li:nth-child(11)'
CSS_MONITOR = 'body > div.page > div:nth-child(8) > div > div.column__main > section:nth-child(6) > div > ul > li:nth-child(12)'
CSS_ETC = 'body > div.page > div:nth-child(8) > div > div.column__main > section:nth-child(6) > div > ul > li:nth-child(13)'
CSS_SHEAT_AIR = 'body > div.page > div:nth-child(8) > div > div.column__main > section:nth-child(6) > div > ul > li:nth-child(18)'
CSS_SHEAT_HEATER = 'body > div.page > div:nth-child(8) > div > div.column__main > section:nth-child(6) > div > ul > li:nth-child(19)'
CSS_LEATHER_SHEAT = 'body > div.page > div:nth-child(8) > div > div.column__main > section:nth-child(6) > div > ul > li:nth-child(22)'
CSS_IDLINGSTOP = 'body > div.page > div:nth-child(8) > div > div.column__main > section:nth-child(6) > div > ul > li:nth-child(23)'
CSS_AS_SENSOR = 'body > div.page > div:nth-child(8) > div > div.column__main > section:nth-child(6) > div > ul > li:nth-child(24)'
CSS_CRUISE = 'body > div.page > div:nth-child(8) > div > div.column__main > section:nth-child(6) > div > ul > li:nth-child(25)'
CSS_ABS = 'body > div.page > div:nth-child(8) > div > div.column__main > section:nth-child(6) > div > ul > li:nth-child(26)'
CSS_ESC = 'body > div.page > div:nth-child(8) > div > div.column__main > section:nth-child(6) > div > ul > li:nth-child(27)'
CSS_ANTI_THEFT = 'body > div.page > div:nth-child(8) > div > div.column__main > section:nth-child(6) > div > ul > li:nth-child(28)'
CSS_AUTO_BRAKE = 'body > div.page > div:nth-child(8) > div > div.column__main > section:nth-child(6) > div > ul > li:nth-child(29)'
CSS_PARKING_ASSIST = 'body > div.page > div:nth-child(8) > div > div.column__main > section:nth-child(6) > div > ul > li:nth-child(30)'
CSS_AIRBAG = 'body > div.page > div:nth-child(8) > div > div.column__main > section:nth-child(6) > div > ul > li:nth-child(31)'
CSS_HEADLIGHT = 'body > div.page > div:nth-child(8) > div > div.column__main > section:nth-child(6) > div > ul > li:nth-child(32)'
CSS_CAMERA = 'body > div.page > div:nth-child(8) > div > div.column__main > section:nth-child(6) > div > ul > li:nth-child(33)'
CSS_AROUND_CAMERA = 'body > div.page > div:nth-child(8) > div > div.column__main > section:nth-child(6) > div > ul > li:nth-child(34)'
CSS_AERO = 'body > div.page > div:nth-child(8) > div > div.column__main > section:nth-child(6) > div > ul > li:nth-child(37)'
CSS_ALUMI_WHEEL = 'body > div.page > div:nth-child(8) > div > div.column__main > section:nth-child(6) > div > ul > li:nth-child(38)'
CSS_LOWDOWN = 'body > div.page > div:nth-child(8) > div > div.column__main > section:nth-child(6) > div > ul > li:nth-child(39)'
CSS_LIFTUP = 'body > div.page > div:nth-child(8) > div > div.column__main > section:nth-child(6) > div > ul > li:nth-child(40)'
CSS_COLD_AREA = 'body > div.page > div:nth-child(8) > div > div.column__main > section:nth-child(6) > div > ul > li:nth-child(41)'


# EMスペース、ノーブレークスペースの符号
EM_SPACE = '\u3000'
NBSP = '\xa0'


parameters = {
    "AR":34,      # Chiba
    "PMAX":800000, # Max price
    "SMAX":70000, # travelled distance
    "YMIN":2014,  
    "SP":"H"      # Hybrid
}

model_dict = {
    'FIT': 'bHO/s028/',
    'SHUTTLE': 'bHO/s107/',
    'GRACE': 'bHO/s103/',
    'JADE': 'bHO/s105/',
    'FIELDER': 'bTO/s046/',
    'VITZ': 'bTO/s018/',
    'AQUA': 'bTO/s228/',
    'NOTE': 'bNI/s008/'
}




URL_DICT = {
    'FIT': 'https://www.carsensor.net/usedcar/bHO/s028/index.html',
    'FIT:CHIBA:HYBRID': 'https://www.carsensor.net/usedcar/bHO/s028/a34/index.html?OPTCD=ETC1&PMAX=1400000&SMAX=70000&SP=H',
    'SHUTTLE': 'https://www.carsensor.net/usedcar/bHO/s107/index.html',
    'SHUTTLE:CHIBA':       'https://www.carsensor.net/usedcar/index.html?STID=CS210610&AR=34&CARC=HO_S107',
    'SHUTTLE:CHIBA:p80max': 'https://www.carsensor.net/usedcar/search.php?STID=CS210610&CARC=HO_S107&AR=34&PMAX=800000',
    'FIELDER': 'https://www.carsensor.net/usedcar/bTO/s046/spH/index.html',
    'FIELDER:CHIBA:p80max': 'https://www.carsensor.net/usedcar/search.php?STID=CS210610&CARC=TO_S046&AR=34&PMAX=800000&',
    'FIELDER:CHIBA:p80max:s70max:ymin2014':'https://www.carsensor.net/usedcar/search.php?STID=CS210610&CARC=TO_S046&AR=34&PMAX=800000&SMAX=70000&YMIN=2014'
}

## Code 

In [158]:
parameters = {
    "AR": 34,  # Chiba
    "PMAX": 900000,  # Max price
    "SMAX": 100000,  # travelled distance
    "YMIN": 2013,
    "SP": "H"  # Hybrid
}
cars = None
for model in model_dict.keys():
    URL = getURL(model, **parameters)
    print(URL, end="")
    try:
        pages = get_lastpage_num(URL)
    except IndexError:
        pages = 1
    print("  ", model, "  {} pp.".format(pages), end="")
    for i in range(pages):
        listpage_url = URL.replace('index', 'index' + str(i))
        urls = scrape_list_page(listpage_url)
        cars_ = get_details(urls)
        if cars_ is not None:
            print(" {}".format(cars_.shape[0]), end="")
            cars_.loc[:, 'model'] = model
            if cars is None:
                cars = cars_
            else:
                cars = pd.concat([cars, cars_], ignore_index=True)
        time.sleep(1)
        print("")

rawdf = cars.copy()

https://www.carsensor.net/usedcar/bHO/s028/a34/index.html?AR=34&PMAX=900000&SMAX=100000&YMIN=2013&SP=H   FIT   1 pp. 30
https://www.carsensor.net/usedcar/bHO/s107/a34/index.html?AR=34&PMAX=900000&SMAX=100000&YMIN=2013&SP=H   SHUTTLE   1 pp. 3
https://www.carsensor.net/usedcar/bHO/s103/a34/index.html?AR=34&PMAX=900000&SMAX=100000&YMIN=2013&SP=H   GRACE   1 pp. 4
https://www.carsensor.net/usedcar/bHO/s105/a34/index.html?AR=34&PMAX=900000&SMAX=100000&YMIN=2013&SP=H   JADE   1 pp. 1
https://www.carsensor.net/usedcar/bTO/s046/a34/index.html?AR=34&PMAX=900000&SMAX=100000&YMIN=2013&SP=H   FIELDER   1 pp. 10
https://www.carsensor.net/usedcar/bTO/s018/a34/index.html?AR=34&PMAX=900000&SMAX=100000&YMIN=2013&SP=H   VITZ   1 pp. 1
https://www.carsensor.net/usedcar/bTO/s228/a34/index.html?AR=34&PMAX=900000&SMAX=100000&YMIN=2013&SP=H   AQUA   1 pp. 30
https://www.carsensor.net/usedcar/bNI/s008/a34/index.html?AR=34&PMAX=900000&SMAX=100000&YMIN=2013&SP=H   NOTE   1 pp. 2


In [159]:
date = datetime.datetime.now()
date_str = date.strftime("%Y%m%d_%H%M")
csv_name = "raw{}.csv".format(date_str)
csv_path = os.path.join('data',csv_name)
rawdf.to_csv(csv_path,index=False)
print("Saved",csv_path)

Saved data/raw20210321_2028.csv


In [160]:
cars = pd.read_csv(csv_path)
cars.base_price = cars.base_price / 10000.
cars.loc[:, 'ref_price'] = cars['base_price']
minref = cars.ref_price.min()
cars.loc[:, 'ref_price'] = cars['ref_price'] - minref + 2  # Min marker size 2
maxtotal = cars['total_price'].max()
cars['total_price'] = cars['total_price'].fillna(maxtotal * 1.1)
cars.total_price = cars.total_price / 10000.
cars.model_year = cars.model_year.astype(str)
print("URLS")
for i, r in cars.iterrows():
    print(i, r['url'])

URLS
0 https://www.carsensor.net/usedcar/detail/VU3428859602/index.html?TRCD=200002&RESTID=CS210610
1 https://www.carsensor.net/usedcar/detail/VU5509373241/index.html?TRCD=200002&RESTID=CS210610
2 https://www.carsensor.net/usedcar/detail/VU5534939755/index.html?TRCD=200002&RESTID=CS210610
3 https://www.carsensor.net/usedcar/detail/VU5293012705/index.html?TRCD=200002&RESTID=CS210610
4 https://www.carsensor.net/usedcar/detail/VU4412965298/index.html?TRCD=200002&RESTID=CS210610
5 https://www.carsensor.net/usedcar/detail/VU5521083324/index.html?TRCD=200002&RESTID=CS210610
6 https://www.carsensor.net/usedcar/detail/VU5016869861/index.html?TRCD=200002&RESTID=CS210610
7 https://www.carsensor.net/usedcar/detail/VU5270314806/index.html?TRCD=200002&RESTID=CS210610
8 https://www.carsensor.net/usedcar/detail/VU5473760859/index.html?TRCD=200002&RESTID=CS210610
9 https://www.carsensor.net/usedcar/detail/VU5299189283/index.html?TRCD=200002&RESTID=CS210610
10 https://www.carsensor.net/usedcar/detail/V

In [177]:
fig = px.scatter(cars,
                 x='distance',
                 size='ref_price',
                 size_max=15,
                 color='model_year',
                 category_orders = {'model_year':['2013','2014','2015','2016','2017','2018','2019','2020']},
                 y='base_price',
                 facet_col='model',
                 facet_col_wrap = 4,
                 template='plotly_dark',
                 symbol='repare',
                 hover_data=[cars.index,'base_price'],
                 color_discrete_sequence = ['#ee0000','#ff6600','#eedd00','#99ff00','#33bb33','#00bb77','0055ff'],
                 color_continuous_scale=px.colors.sequential.Turbo,
                height=600,
                width=1200)
# fig.update(layout_coloraxis_showscale=False)
fig.show()

In [87]:
cars.iloc[0]

url                  https://www.carsensor.net/usedcar/detail/VU550...
base_price                                                      678000
total_price                                                   828000.0
model_year                                                        2014
distance                                                           6.4
repare                                                              なし
inspection                                                       車検整備無
info                 1.5 ハイブリッド Sパッケージ 衝突軽減/ナビ/Bカメ/Bluetooth/ETC/LE...
drive                                                              2WD
recycle                                                            リ済別
legal_maintenance                                                法定整備無
warranty                                    保証付：販売店保証 保証期間：1年 保証距離：無制限
one_owner                                                            0
record_book                                                          0
no_smo